# Homework 3

## Count Word Bigrams

### 1

In [11]:
def count_bigrams_in_file(filename):
    if type(filename) is not str:
        raise TypeError("Input is not string.")
    else:
        try:
            f = open(filename)
        except:
            print("Wrong filename: file ", filename, " can't be opened.")
    words = list()
    chars = ['*','"','?','.','!',':',';',')','(',',','-','@','#','$','^','%','&','/','}','{','[',']','+']
    numbers = ['0','1','2','3','4','5','6','7','8','9']
    for line in f:
        for i in chars:
            line = line.replace(i," ")
        for wd in line.split():
            wd = wd.strip("'")
            wd = wd.strip('"')
            r = 1
            for j in numbers:
                r *= -(wd.find(j))
            if r != 0:
                words.append(wd.lower())
    hist = dict()
    for i in range(len(words)-1): 
        d = tuple(words[i:i+2])
        hist[d] = hist.get(d,0) + 1
    f.close()
    return hist
count_bigrams_in_file("poem.txt")

{('a', 'league'): 3,
 ('all', 'in'): 1,
 ('death', 'rode'): 1,
 ('half', 'a'): 3,
 ('in', 'the'): 1,
 ('league', 'half'): 2,
 ('league', 'onward'): 1,
 ('of', 'death'): 1,
 ('onward', 'all'): 1,
 ('rode', 'the'): 1,
 ('six', 'hundred'): 1,
 ('the', 'six'): 1,
 ('the', 'valley'): 1,
 ('valley', 'of'): 1}

### 2

In [12]:
import pickle
hist = count_bigrams_in_file("WandP.txt")
p = pickle.dumps(hist)
f = open('mb.bigrams.pickle','wb')
f.write(p)
f.close()

### 3

In [13]:
def collocations(filename):
    hist_bigrams = count_bigrams_in_file(filename)
    list_bigrams = list(hist_bigrams.keys())
    hist = dict()
    for bigram in list_bigrams:
        a = bigram[0]
        b = bigram[1]
        c = hist.get(a,set())
        c.add(b)
        hist[a] = c
        c = hist.get(b,set())
        c.add(a)
        hist[b] = c
    return hist
print(collocations("poem.txt"))
    

{'half': {'league', 'a'}, 'six': {'the', 'hundred'}, 'a': {'half', 'league'}, 'hundred': {'six'}, 'rode': {'the', 'death'}, 'of': {'death', 'valley'}, 'onward': {'all', 'league'}, 'league': {'half', 'onward', 'a'}, 'death': {'rode', 'of'}, 'all': {'onward', 'in'}, 'in': {'all', 'the'}, 'valley': {'the', 'of'}, 'the': {'rode', 'six', 'in', 'valley'}}


### 4

In [14]:
hist = collocations('WandP.txt')
p = pickle.dumps(hist)
f = open('mb.colloc.pickle','wb')
f.write(p)
f.close()

## 2. More Fun with Vectors

In [15]:
class Vector:
    '''Representation of vectors with dimension and elements in a tuple'''
    
    def __init__(self, dim = 1, args = ()): ## 1. Initialization: dimension must match with the number of elements given
        if type(dim) is not int:
            raise TypeError("Dimension must be integer")
        elif dim <= 0:
            raise ValueError("Dimension must be positive integer.")
        else:
            self.dim = dim
            if len(args) == 0:
                self.vector = tuple([0]*dim) ## If args is empty, for a valid dimension, we define the entries to be the 
                ## (0,0,...) tuple
            else:
                if type(args) is not tuple:
                    try:
                        args = tuple(args)
                    except:
                        raise TypeError("args is not sequence")
                for x in args:
                    if type(x) is not (int or float):
                        raise TypeError("Some of the values in args is neither int nor float")
                if len(args) != self.dim:
                    raise ValueError("Number of arguments don't match with dimension.")
                self.vector = args
        
        
    def __lt__(self,other) : ## 4: Implementation of comparison operators
        if type(other) is not Vector:
            raise TypeError("other is not a Vector")
        if self.dim != other.dim:
            raise ValueError("Dimensions are different. Comparison is not possible")
        else:
            return self.vector < other.vector
    
    def __gt__(self,other):
        return other.lt(self)
    
    def __eq__(self,other):
        if type(other) is not Vector:
            raise TypeError("other is not a Vector")
        if self.dim != other.dim:
            raise ValueError("Dimensions are different. Comparison is not possible")
        else:
            return self.vector == other.vector
            
    def __le__(self,other):
        return (self < other) or (self == other)
    
    
    def __ge__(self,other):
        return other.le(self)
    
    def dot(self, other): ## 5. Implementation of dot product
        if type(other) is not Vector:
            raise TypeError("other is not a vector")
        if self.dim != other.dim:
            raise ValueError("Dimensions are not equal")
        s = 0
        for i in range(self.dim):
            s += self.vector[i]*other.vector[i]
        return s
    
    
    def __mul__(self,other): ## 6. Implementation of vector multiplication
        if type(other) is (int or float):
            t = [other*a for a in self.vector]
            return Vector(dim = self.dim, args = t)
        elif type(other) is Vector:
            if other.dim != self.dim:
                raise ValueError("Dimensions don't match")
            else:
                t = [self.vector[i]*other.vector[i] for i in range(self.dim)]
                return Vector(dim = self.dim, args = t)
        else:
            raise TypeError("other is not a vector") 
            
    def __rmul__(self,other):
        return self.__mul__(other)
    
    import math
    def norm(self,p):  ##7: Implementation of norm function
        if p == math.inf:
            ret = 0
            for i in range(self.dim):
                if abs(self.vector[i])>ret:
                    ret = self.vector[i]
            return ret 
        elif p == -math.inf:
            raise ValueError("p is negative infinity.")
        else:
            if type(p) is not (int or float):
                raise TypeError("p is neither int nor float.")
            elif p<0:
                raise ValueError("p is negative.")
            elif p==0:
                s = 0
                for i in range(self.dim):
                    if self.vector[i] != 0:
                        s += 1
                return s
            else:
                ret = 0
                for i in range(self.dim):
                    ret += pow(self.vector[i],p)
                ret = pow(ret,1/p)
                return ret
            

### 2. 
I chose vector entries to be tuple. Since tuples are immutable, they require less memory. We can define large tuple, which will help us to define large vectors. 

### 3.
Dimensions and entries are instance attributes. Since different instances of class "Vector" might have different dimensions and entries, instance attributes allows us to have different values for them, for different instances. If they were defined as class attributes, then all the instances of that class would have same value of that attribute. 